# Car Detection

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/car_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
# %pip install geoai-py

## Import libraries

In [ ]:
import geoai

## Download sample data

We will download a sample image from Hugging Face Hub to use for car detection. You can find more high-resolution images from [OpenAerialMap](https://openaerialmap.org).

In [ ]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_7cm.tif"
)

In [ ]:
raster_path = geoai.download_file(raster_url)

## Visualize the image

In [ ]:
geoai.view_raster(raster_url)

## Initialize the model

In [ ]:
detector = geoai.CarDetector()

## Extract cars

Extract cars from the image using the model and save the output image.

In [ ]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="cars_masks.tif",
    confidence_threshold=0.3,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(400, 400),
)

Convert the image masks to polygons and save the output GeoJSON file.

In [ ]:
gdf = detector.vectorize_masks(
    masks_path="cars_masks.tif",
    output_path="cars.geojson",
    min_object_area=100,
    max_object_area=2000,
)

## Add geometric properties

In [ ]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize initial results

In [ ]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter cars by area

In [ ]:
gdf_filter = gdf[
    (gdf["area_m2"] > 8) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [ ]:
len(gdf_filter)

## Visualiza final results

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

In [ ]:
geoai.view_vector_interactive(gdf_filter, tiles=raster_url)

![image](https://github.com/user-attachments/assets/a1e4c871-b152-466a-b902-7c00e62e5444)